# Tukey's multiple comparisons tests

The Tukey Honest Significant Difference (HSD) test is a widely used post hoc analysis method for identifying which specific group means differ significantly after obtaining a statistically significant result from an ANOVA. While ANOVA can tell us that at least one group is different, it doesn't indicate which ones. The pairwise_tukeyhsd function in statsmodels fills this gap by performing all possible pairwise comparisons between group means while controlling the family-wise error rate. This makes it especially valuable when analysing categorical factors with three or more levels, allowing for detailed interpretation of group differences while maintaining statistical rigour.

In [ ]:
from IPython.display import display, Markdown
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import numpy as np

## Generate random data

First, we will generate synthetic data by sampling from normal distributions. Specifically, we will create 8 groups, each containing 30 samples drawn from a normal distribution with a fixed variance of 1. The group means will vary to ensure that some groups differ significantly from others, allowing us to demonstrate the utility of post hoc analysis.

In [ ]:
data = []
groups = []
locs = [0, 0.1, 1, 1.2, 1.5, 1.7, 2.0, 3.0]
np.random.seed(0)
for i in range(8):
    group_data = np.random.normal(loc=locs[i], scale=1.0, size=30)
    data.extend(group_data)
    groups.extend([f'Group {i+1}'] * 30)

## ANOVA I test

Before applying the Tukey HSD test, we will first perform a one-way ANOVA to determine whether there are any statistically significant differences among the group means. ANOVA serves as an initial global test, assessing whether at least one group differs from the others. If the ANOVA result is significant, we can proceed with the Tukey HSD test to identify which specific pairs of groups are significantly different.

In [ ]:
anova_model = ols('value ~ C(group)', data={'value': data, 'group': groups}).fit()
anova_results = anova_lm(anova_model, typ=2)
display(anova_results)

## Pairwise Tukey HSD test

Since our factor has more than two levels, a significant ANOVA result only tells us that at least one group differs from the others, but not which ones. To investigate these differences in more detail, we will use the Tukey HSD test, which performs all pairwise comparisons between group means while controlling for the increased risk of Type I error due to multiple testing. This allows us to determine exactly which group pairs show statistically significant differences.

In [ ]:
alpha = 0.05

tukey = pairwise_tukeyhsd(data, groups, alpha=alpha)
print(tukey)

### Tukey summary frame

Using `tukey.summary_frame()` we can display a dataframe with the result of all the pairwise comparisons.

In [ ]:
display(tukey.summary_frame())

### Tukey Homogeneous Subsets

Also, using `tukey.create_homogeneous_subsets_dataframe()`, we can display a summary table of the test results.

This function constructs a DataFrame that groups factor levels into **homogeneous subsets** based on the results of the Tukey HSD test. Groups are considered part of the same subset if their pairwise differences are **not statistically significant** (i.e., *p* > alpha). Each group appears once in the table, with its mean shown under every subset to which it belongs. At the bottom of the table, a **"min p-value"** row shows the smallest p-value among all pairwise comparisons within each subset. This provides a clear visual summary of which groups are statistically similar, helping to interpret the post hoc results at a glance.


In [ ]:
hs_df = tukey.create_homogeneous_subsets_dataframe()
display(hs_df.fillna('')) # Fill NaN with empty strings for better display

### Tukey simultaneous plot

To visualise group mean differences in a more interpretable way, we can use the `plot_simultaneous()` method to create a **universal confidence interval plot**. Instead of displaying all pairwise confidence intervals (which can be overwhelming with many groups), this method shows a **single confidence interval for each group mean**.

This approach, originally proposed by Hochberg and Tamhane (1987), uses Tukey's Q critical value to calculate interval widths, allowing us to compare any two group means visually. If the confidence intervals of two groups **do not overlap**, the difference between them is statistically significant.

In [ ]:
tukey.plot_simultaneous()
pass